In [4]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
%autosave 0

Autosave disabled


# WeightWatcher Forschung

> WeightWatcher (w|w) is an open-source, diagnostic tool for analyzing Deep Neural Networks (DNN),
> without needing access to training or even test data. It is based on theoretical research into
> Why Deep Learning Works, based on our Theory of Heavy-Tailed Self-Regularization (HT-SR)... <sup>[1]</sup>

## Hintergrund

Beim Evaluieren der Qualität eines DNN gibt es mehrere Ansätze. Ein sehr verbreiteter Ansatz
ist das Auswerten der von dem Netzwerk, oft auch *Modell* genannt, vorhersagten Ergebnisse.
Die Ergebnisse werden dabei anhand einer für das Aufgabenfeld des Modells typischen
Metrik bewertet. Dieser Ansatz birgt jedoch einige Probleme, wie beispielweise die dafür
benötigten Ressourcen und die aussagekraft in Bezug auf den Grad der Generalisierung eines
Modells.

Als Alternative zu diesem Ansatz bieten sich statische Analyseverfahren an. Diese werden verwendet,
um die Parameter der Modelle zu untersuchen, anstatt der von den Modellen erzielten Ergebnisse.
Für die Bewertung der Parameter müssen jedoch andere, teilweise auch weniger intuitive, Metriken
verwendet werden.

An dieser Stelle kommt das sogenannte *WeightWatcher* Tool **WeightWatcher** zum Einsatz.
Mit diesem werden *Convolutional Layers* und *Fully-connected Layers* auf ihre
Generalisierungsfähigkeit untersucht. Im Grunde werden dabei die Eigenwerte der Gewichtsmatrizen
der Layer mit Modellen der *Random Matrix Theory* und anderen Bereichen der mathematischen Physik
analysiert. Die dabei entwickelte Metriken wird **Alpha $\alpha$** genant. Mithilfe dieser lassen
sich Aussagen über neurale Netzwerke treffen, die in Zusammenhang mit der Generalisierungsfähigkeit
stehen. Beispiele dafür finden sich auf der offiziellen Webseite des Tools.<sup>[1]</sup>

## Zielstellung

*WeightWatcher* hat aber auch gewisse Einschränkungen. Zunächst einmal sind die auf den Eigenwerten
von Matrizen beruhenden Berechnung bei größeren Matrizen aussagekräftiger. Daher funktioniert das Tool
bei sehr großen Layer mit vielen Gewichten und großen Gewichtsmatrizen am besten. Außerdem ist der
Verwendung von WeightWatcher aufgefallen, dass es einige Architekturen von neuronalen Netzwerken gibt,
bei denen die relevanten Metriken nicht oder sogar negativ mit Alpha korrelieren.
Aus diesem Grund wird die Forschungsfrage "Welchen Einfluss hat die Architektur eines neuronalen
Netzwerks auf die Korrelation von Alpha Werten und Qualitätsmetriken?" formuliert. Die bei der
Beantwortung der Forschungsfrage gewonnen Erkenntnisse werden im Folgenden präsentiert.

Das Ziel dieser Forschung ist die Beantwortung der Forschungsfrage: "Inwiefern beeinflusst die
Architektur eines neuronalen Netzwerks die Aussagekraft des Alpha-Werts?"
1. How well does the WeightWatcher metric "alpha" work when comparing
    - different variants of different architecture?
    - different variants of the same architecture?
    - differently trained models of the same variant and architecture?
2. What are reasonable explanations for those observations?

## Datenerhebung

### Keras Model API

### Pytorch Image Models (timm)

### Ergebnisse der Entwickler von WeightWatcher

In [5]:
from weight_watcher import WeightWatcherResultService

keras_result_repository = WeightWatcherResultService('results')
keras_results = keras_result_repository.load_all()

**TODO**:
* Datenerhebung durch Pytorch Image Models (timm)
* Datenerhebung durch die Ergebnisse der Entwickler von WeightWatcher

## Visualisierung

In [6]:
from visualization import PlottingService, DashboardService

from typing import List
import plotly.graph_objects as go

plotting_service = PlottingService()
dashboard_service = DashboardService()

plots: List[go.Figure] = []
plots += [plotting_service.create_summaries_figure(keras_results, group_by_variant=False)]
plots += plotting_service.create_summaries_per_architecture_figures(keras_results)
plots += [plotting_service.create_details_figure(keras_results, group_by_architecture=True, group_by_variant=False)]
plots += plotting_service.create_details_per_architecture_figures(keras_results)
plots += plotting_service.create_details_per_model_figures(keras_results)

dashboard_service.build_dashboard(plots)
dashboard_service.show_dashboard(True)

Dash app running on http://127.0.0.1:8050/


**TODO:**
* Visualisierung der Architekturen?

## Diskussion

## Quellen

[1]: <cite>[WeightWatcher Homepage](https://weightwatcher.ai/#Research)</cite>